# Infinite Investment Systems Challenge

### The Challenge

The challenge involves analyzing a comprehensive dataset on customer behavior within the wealth management services domain. The dataset, consisting of many columns, aggregates information from various tables, offering insights into customer interactions, financial transactions, trading activities, and overall financial behaviors.

The primary focus is on predicting customer churn and estimating lifetime value.



In [ ]:
import pandas as pd

In [ ]:
!pip install --upgrade numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Copy Dataset into your Google Drive
1. Right Click `history.csv` and select "Make a Copy". This will create a copy of the dataset into your drive
2. **Make sure to rename the copy from "Copy of history.csv" to "history.csv"**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/history.csv", low_memory=False)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9d in position 9551: invalid start byte

In [ ]:
df.head(10)
print(account_info.head())

Preprocessing the data above...

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns.drop('label')  # Exclude the target variable

for col in categorical_cols:
    df[col] = df[col].astype(str)

# Preprocessing for numerical data
# numerical_transformer = SimpleImputer(strategy='median')
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


Will use RandomForestClassifier, as it seems to be the best fit for this data! Used XGBoost, didn't work too well..

In [ ]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Assuming y_train is your target variable for the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

# Define the model with class weights
model = RandomForestClassifier(n_estimators=100, max_depth=10, max_leaf_nodes=16, n_jobs=-1, random_state=0, class_weight=class_weights_dict)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Split data into train and test sets
X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Preprocessing of training data, fit model
clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'class_id', 'contract_type', 'cashflows_custody_fee',
       'credit_limit_type', 'dividend_confirm_code', 'options_trading_type',
       'rep_commission_rate', 'rep_commission_override',
       'interest_...
       'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp',
       'is_plan_grandfathered', 'inserted_at', 'updated_at', 'is_olob',
       'retail_last_maintenance_time', 'retail_last_maintenance_user',
       'visible_in_reports'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(class_weight={'Churn': 0.6252855550912383,
                                                      'No Churn': 2.4954415320899463},
                                        max_depth=10, max_leaf_nodes=16,
                                        n_jobs=-1, random_state=0))])

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score

# Predictions on test data
preds_test = clf.predict(X_test)

# Calculate metrics
print(classification_report(y_test, preds_test))

# Cross-validation
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print(f"Average ROC AUC Score (cross-validation): {scores.mean()}")

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


              precision    recall  f1-score   support

       Churn       1.00      0.96      0.98    107704
    No Churn       0.85      1.00      0.92     26964

    accuracy                           0.96    134668
   macro avg       0.92      0.98      0.95    134668
weighted avg       0.97      0.96      0.97    134668



/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  war

Average ROC AUC Score (cross-validation): 0.9797153549296391


In [ ]:
pip install --upgrade numpy scikit-learn scikit-optimize

In [ ]:
from sklearn.metrics import f1_score

# First, fit the classifier on the training data
# clf.fit(X_train, y_train)

# Then, predict on the test data
y_pred = clf.predict(X_test)

# Now, you can compute the F1 score with 'Churn' as the positive label
f1 = f1_score(y_test, y_pred, pos_label='Churn')
print(f"F1 Score: {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['cashflows_custody_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


F1 Score: 0.9771573724568525


good F1 score!

cross-validation stuff...

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

# Update the model with class weights
model = RandomForestClassifier(n_estimators=100, max_depth=10, max_leaf_nodes=16, n_jobs=-1, random_state=0, class_weight=class_weights_dict)

In [ ]:
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer

# Assuming 'preprocessor' is your preprocessing pipeline defined earlier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Fit the model to your training data
clf.fit(X_train, y_train)

In [ ]:
#from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Cross-validation with the tuned model
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print(f"Average ROC AUC Score (cross-validation): {scores.mean()}")

# Final evaluation on the test set
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
f1 = f1_score(y_test, y_pred, pos_label='Churn')
print(f"F1 Score: {f1}")


running model on test dataset + storing it in submission.csv!

In [ ]:
import pandas as pd

# Load the test dataset
test_df = pd.read_csv("/content/drive/MyDrive/test.csv", low_memory=False)

# If referring to columns by names rather than indices, ensure they exist
# For example, if your DataFrame uses specific names, replace '62', '64', '87' with actual column names
column_names = test_df.columns  # Get all column names from test_df
columns_to_convert = [col for col in ['62', '64', '87'] if col in column_names]  # Ensure they exist in test_df

# Convert identified columns to numeric, handling non-numeric values
for col in columns_to_convert:
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# Assuming features_used_during_training correctly lists all features used in the model
# Ensure this list is defined based on your training data preparation
features_used_during_training = X_train.columns.tolist()  # Make sure X_train reflects your training features accurately

# Prepare test_df with only the features used during training
test_df_prepared = test_df[features_used_during_training]

# Convert categorical columns to string if done so during training
categorical_cols = [col for col in test_df_prepared.columns if test_df_prepared[col].dtype == 'object']  # Adjust based on actual categorical features handling
for col in categorical_cols:
    test_df_prepared[col] = test_df_prepared[col].astype(str)

# Predict using the prepared test data
predictions = clf.predict(test_df_prepared)

# Create and save the submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Confirm 'id' is the correct identifier column
    'predicted': predictions
})
submission_df.to_csv("/content/drive/MyDrive/submission.csv", index=False)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'A'